In [40]:
import full_model
import fast_model_improved
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from joblib import Parallel, delayed

## This note book aims to discover some properties of deinfluencers in restricted setting

In [29]:
# Generate the graph
# G = nx.erdos_renyi_graph(100, 0.1, directed=True)
# G = nx.watts_strogatz_graph(100, 4, 0.1).to_directed()
G = nx.barabasi_albert_graph(500, 2).to_directed()

# Initialize the InfluenceDeinfluenceModel
model = fast_model_improved.InfluenceDeinfluenceModel(G,"fixed")

# number of influencers
k = 5

# number of steps
steps = 3
influencers = model.greedy_hill_climbing(k, steps)

model.reset_graph()
#model.set_initial_states()

print("Influencers:", influencers)

Influencers: {385, 3, 36, 5, 43}


Validate that the code works as intended. Should only be used to when step = 1 and number of influencer = 2.

In [26]:
influencers = list(influencers)
node1 = influencers[0]
node2 = influencers[1]

model.set_influencers(influencers)
model.run_cascade(steps)
k = model.evaluate_influence()
print(k)
print(G.out_degree(node1))
print(G.out_degree(node2))

node_x_out_neighbors = set(G.successors(node1))
node_y_out_neighbors = set(G.successors(node2))
print("Node y out-neighbors:", list(node_y_out_neighbors))

common_out_neighbors = node_x_out_neighbors & node_y_out_neighbors
print("Common out-neighbors:", list(common_out_neighbors))

# Corrected condition
if k - (G.out_degree(node1) + G.out_degree(node2) + 2 - len(common_out_neighbors)) == 0:
    print("yes")
else:
    print("no")

model.reset_graph()

500
46
7
Node y out-neighbors: [320, 66, 419, 459, 461, 143, 383]
Common out-neighbors: []
no


# Examine how the change in constant c influence final inf/deinf

In [46]:
# Generate the graph
# G = nx.erdos_renyi_graph(100, 0.1, directed=True)
# G = nx.watts_strogatz_graph(100, 4, 0.1).to_directed()
G = nx.barabasi_albert_graph(300, 3).to_directed()

# Initialize the InfluenceDeinfluenceModel
model = fast_model_improved.InfluenceDeinfluenceModel(G,"fixed")

# number of influencers
k = 10

# number of steps
steps = 4
influencers = model.greedy_hill_climbing(k, steps)


model.reset_graph()
#model.set_initial_states()

print("Influencers:", influencers)

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0

Influencers: {3, 293, 6, 105, 111, 150, 123, 28, 126, 31}


In [45]:
steps = 5

ls = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]    

G = nx.barabasi_albert_graph(300, 3).to_directed()
model = fast_model_improved.InfluenceDeinfluenceModel(G, 'dominate', c=1)
model.selected_influencers = model.random_influencers(10)
model.selected_deinfluencers = model.random_deinfluencers(5)

print("Selected influencers:", model.selected_influencers)
print("Selected deinfluencers:", model.selected_deinfluencers)

model.set_influencers(model.selected_influencers)
model.set_deinfluencers(model.selected_deinfluencers)

model.run_cascade(steps)

k = model.evaluate_influence()
j = model.evaluate_deinfluence()
print("Initial run - Influence:", k, "Deinfluence:", j)
print("_____")


for i in range(len(ls)):
    model_f = fast_model_improved.InfluenceDeinfluenceModel(G, 'dominate', c=ls[i])
    model_f.reset_graph()
    model_f.set_influencers(model.selected_influencers)
    model_f.set_deinfluencers(model.selected_deinfluencers)

    model_f.run_cascade(steps)
    k = model_f.evaluate_influence()
    j = model_f.evaluate_deinfluence()
    print("For c =", ls[i], "- Influence:", k, "Deinfluence:", j)

Selected influencers: {195, 4, 26, 11, 209, 115, 244, 246, 218, 63}
Selected deinfluencers: {35, 105, 243, 180, 85}
Initial run - Influence: 2 Deinfluence: 297
_____
For c = 0.1 - Influence: 273 Deinfluence: 25
For c = 0.2 - Influence: 160 Deinfluence: 135
For c = 0.3 - Influence: 123 Deinfluence: 172
For c = 0.4 - Influence: 100 Deinfluence: 198
For c = 0.5 - Influence: 54 Deinfluence: 240
For c = 0.6 - Influence: 17 Deinfluence: 278
For c = 0.7 - Influence: 8 Deinfluence: 288
For c = 0.8 - Influence: 15 Deinfluence: 280
For c = 0.9 - Influence: 5 Deinfluence: 295
For c = 1 - Influence: 3 Deinfluence: 296
For c = 1.1 - Influence: 4 Deinfluence: 293
For c = 1.2 - Influence: 6 Deinfluence: 294
For c = 1.3 - Influence: 1 Deinfluence: 299
For c = 1.4 - Influence: 1 Deinfluence: 299
For c = 1.5 - Influence: 0 Deinfluence: 300
For c = 1.6 - Influence: 1 Deinfluence: 299
For c = 1.7 - Influence: 1 Deinfluence: 299
For c = 1.8 - Influence: 0 Deinfluence: 298
For c = 1.9 - Influence: 0 Deinflu